# Scraping College Football Data

In [2]:
import pandas as pd
import numpy as np
import requests
import cfbd
import json

Below is how I authorized access to the API at [CollegeFootballData.com](CollegeFootballData.com) and ran a preliminary GET request to collect data on college football conferences using the `cfbd` package

In [3]:
# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'yUuTebpXwMv5lRf4LsKphgt1JSyr3zX/BgrmOmJIZLE+6FC04u+1Ur6TUNaRXlo0'
configuration.api_key_prefix['Authorization'] = 'Bearer'

In [7]:
# create an instance of the API class
api_instance = cfbd.ConferencesApi(cfbd.ApiClient(configuration))
api_response = (api_instance.get_conferences())
api_response

[{'abbreviation': 'ACC',
 'id': 1,
 'name': 'ACC',
 'short_name': 'Atlantic Coast Conference'}, {'abbreviation': 'B12',
 'id': 4,
 'name': 'Big 12',
 'short_name': 'Big 12 Conference'}, {'abbreviation': 'B1G',
 'id': 5,
 'name': 'Big Ten',
 'short_name': 'Big Ten Conference'}, {'abbreviation': 'SEC',
 'id': 8,
 'name': 'SEC',
 'short_name': 'Southeastern Conference'}, {'abbreviation': 'PAC',
 'id': 9,
 'name': 'Pac-12',
 'short_name': 'Pac-12 Conference'}, {'abbreviation': 'CUSA',
 'id': 12,
 'name': 'Conference USA',
 'short_name': 'Conference USA'}, {'abbreviation': 'MAC',
 'id': 15,
 'name': 'Mid-American',
 'short_name': 'Mid-American Conference'}, {'abbreviation': 'MWC',
 'id': 17,
 'name': 'Mountain West',
 'short_name': 'Mountain West Conference'}, {'abbreviation': 'Ind',
 'id': 18,
 'name': 'FBS Independents',
 'short_name': 'FBS Independents'}, {'abbreviation': 'SBC',
 'id': 37,
 'name': 'Sun Belt',
 'short_name': 'Sun Belt Conference'}, {'abbreviation': 'AAC',
 'id': 151,
 'n

Upon browsing the [cfbd documentation](https://github.com/CFBD/cfbd-python/tree/master/cfbd/models/conference.py), I found that there is a `self.to_dict()` method which we can leverage for creating data frames. It seems like each entry in our response is already a dictionary but it is actually a `models.conference` instance:

In [8]:
type(api_response[0])

<class 'cfbd.models.conference.Conference'>

In [9]:
updated_response = [conference.to_dict() for conference in api_response]
conference_data = pd.DataFrame(updated_response, columns=['id', 'name', 'abbreviation', 'short_name'])
conference_data.head()

,id,name,abbreviation,short_name
0,1,ACC,ACC,Atlantic Coast Conference
1,4,Big 12,B12,Big 12 Conference
2,5,Big Ten,B1G,Big Ten Conference
3,8,SEC,SEC,Southeastern Conference
4,9,Pac-12,PAC,Pac-12 Conference


I'm not a big fan of having to convert every entry in the response to a dictionary from a `conference` object, as this could become cumbersome as our data becomes more nested. Here's how we can gather the same response using the `requests` library.

In [10]:
# Using request library instead:
my_session = requests.Session()
auth_key = 'Bearer yUuTebpXwMv5lRf4LsKphgt1JSyr3zX/BgrmOmJIZLE+6FC04u+1Ur6TUNaRXlo0'
base_url = 'https://api.collegefootballdata.com/'
# So we don't have to list our authentication token with every request
my_session.headers.update({'Authorization': auth_key})

In [11]:
conference_url = base_url + 'conferences'
response = my_session.get(conference_url)
response # Double check successful response code

<Response [200]>

In [12]:
conference_data = pd.DataFrame(response.json(), columns=['id', 'name', 'abbreviation', 'short_name'])
conference_data.head()

,id,name,abbreviation,short_name
0,1,ACC,ACC,Atlantic Coast Conference
1,4,Big 12,B12,Big 12 Conference
2,5,Big Ten,B1G,Big Ten Conference
3,8,SEC,SEC,Southeastern Conference
4,9,Pac-12,PAC,Pac-12 Conference


## Scraping Team talent data

In [13]:
talent_url = base_url + 'talent'
years_talent = [i for i in range(2015, 2021)] # Talent is only available after 2015

Because data is being collected on several different years and year is a parameter in the request, we need to make several `GET` requests and concatenate the returned `JSON` objects.

In [14]:
talent_data = pd.DataFrame()
for year in years_talent:
    response = my_session.get(talent_url, params={
            'year': year})
    talent_data = pd.concat([talent_data, pd.DataFrame(response.json())])
talent_data['talent'] = talent_data['talent'].astype(float)
talent_data.sample(10)

,year,school,talent
38,2019,Oklahoma State,644.42
9,2015,Texas A&M,830.34
55,2017,Iowa State,559.34
32,2020,Michigan State,673.43
65,2019,Wake Forest,563.24
0,2017,Alabama,997.57
144,2016,Hampton,99.46
16,2017,Ole Miss,795.70
188,2019,Southern Utah,35.09
216,2019,Alabama A&M,13.54


Out of curiousity, what schools were considered the most talented in our data, and what were their respective talent levels?

In [15]:
talent_data.loc[talent_data['talent'].idxmax()]

,year,school,talent
0,2015,Alabama,981.90
0,2016,Alabama,982.66
0,2017,Alabama,997.57
0,2018,Ohio State,984.30
0,2019,Alabama,984.96
0,2020,Georgia,990.47


Considering Alabama's current dynasty and run of dominance, their presence in this table is not particularly a surprise. Let's check-in on my home team.

In [16]:
talent_data[talent_data['school'] == 'Michigan']

,year,school,talent
8,2015,Michigan,851.12
7,2016,Michigan,852.78
6,2017,Michigan,874.28
7,2018,Michigan,862.35
10,2019,Michigan,853.40
13,2020,Michigan,832.65


Eh, not bad!

Anywaym let's sort our data alphabetically by school, then chronologically. and export the results to csv.

In [17]:
talent_data.sort_values(['school', 'year']).to_csv('talent_data.csv', index=False)

In [18]:
talent_test = pd.read_csv('talent_data.csv')
talent_test

,year,school,talent
0,2015,Abilene Christian,66.43
1,2016,Abilene Christian,19.77
2,2018,Abilene Christian,43.10
3,2019,Abilene Christian,46.84
4,2020,Abilene Christian,79.83
...,...,...,...
1308,2015,Youngstown State,116.79
1309,2016,Youngstown State,130.84
1310,2018,Youngstown State,70.45
1311,2019,Youngstown State,67.72


## Scraping Season Stats

In [19]:
stats_url = base_url + 'stats/season/'

In [335]:
# Concatenating a number of JSON objects and sorting, as before
stats_df = pd.DataFrame()
for year in years:
    response = my_session.get(stats_url, params={'year':year})
    stats_df = stats_df.append(response.json())
stats_df.sort_values(['team', 'season'], inplace=True)

In [336]:
stats_df

,season,team,conference,statName,statValue
28,2015,Air Force,Mountain West,kickReturnTDs,0
120,2015,Air Force,Mountain West,puntReturns,15
288,2015,Air Force,Mountain West,rushingAttempts,816
343,2015,Air Force,Mountain West,puntReturnYards,139
393,2015,Air Force,Mountain West,penalties,53
...,...,...,...,...,...
3340,2020,Wyoming,Mountain West,kickReturnTDs,0
3364,2020,Wyoming,Mountain West,puntReturnYards,66
3563,2020,Wyoming,Mountain West,puntReturns,12
3878,2020,Wyoming,Mountain West,kickReturnYards,290


Now this is not in a usable form (Yet!) We can see that the `statName` and `value` features are not in a *tidy* format, meaning each column is a feature and each row entry is an observation of that feature. Rather, we have a generic column with each entry comprising a feature and it's corresponding value in the cell to it's right. We'll use panda's [`pivot`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html) functionality here.

In [337]:
season_stats = stats_df.pivot(index = ['team', 'season', 'conference'], columns = 'statName', values='statValue').reset_index()
season_stats

statName,team,season,conference,firstDowns,fourthDownConversions,fourthDowns,fumblesLost,fumblesRecovered,games,interceptionTDs,...,puntReturns,rushingAttempts,rushingTDs,rushingYards,sacks,tacklesForLoss,thirdDownConversions,thirdDowns,totalYards,turnovers
0,Air Force,2015,Mountain West,300.0,14.0,23.0,11.0,7.0,14.0,1.0,...,15.0,816.0,45.0,4468.0,NaN,NaN,84.0,196.0,6315.0,23.0
1,Air Force,2016,Mountain West,302.0,8.0,15.0,9.0,8.0,13.0,1.0,...,10.0,827.0,39.0,4126.0,30.0,71.0,99.0,196.0,5878.0,17.0
2,Air Force,2017,Mountain West,290.0,16.0,26.0,14.0,4.0,12.0,1.0,...,12.0,766.0,38.0,3683.0,7.0,38.0,81.0,167.0,5004.0,20.0
3,Air Force,2018,Mountain West,264.0,24.0,39.0,10.0,7.0,12.0,2.0,...,12.0,713.0,33.0,3404.0,12.0,51.0,70.0,169.0,4980.0,15.0
4,Air Force,2019,Mountain West,286.0,16.0,21.0,12.0,14.0,13.0,3.0,...,5.0,755.0,41.0,3881.0,32.0,73.0,84.0,157.0,5483.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,Wyoming,2016,Mountain West,312.0,10.0,18.0,9.0,11.0,14.0,3.0,...,32.0,628.0,31.0,2865.0,30.0,79.0,88.0,202.0,6072.0,24.0
770,Wyoming,2017,Mountain West,198.0,9.0,14.0,6.0,17.0,13.0,2.0,...,24.0,446.0,13.0,1414.0,32.0,87.0,60.0,183.0,3718.0,14.0
771,Wyoming,2018,Mountain West,204.0,5.0,13.0,7.0,5.0,12.0,1.0,...,27.0,495.0,19.0,2395.0,18.0,66.0,63.0,174.0,3971.0,11.0
772,Wyoming,2019,Mountain West,230.0,6.0,12.0,6.0,12.0,13.0,2.0,...,34.0,575.0,27.0,2792.0,44.0,104.0,81.0,188.0,4562.0,15.0


***Exhales*** 

So much better.

Now, I want to change the appropriate categories to season average statistics

*Why?*

Because the point of this product is to answer the question: ***Would team A in year 20XX beat team B in year 20YY?*** Heuristically, it makes sense that season average data is appropriate. I don't think it would be detrimental to the quality of analysis and more often than not, averages are used over totals in sports.

In [ ]:
# Identify the columns that we want to convert
season_stats.columns
cols_to_divide = season_stats.columns.values.tolist()[3:8]
cols_to_divide.extend(season_stats.columns.values.tolist()[10:])

# Divide these columns by the respective number of games a team played that year
season_stats[cols_to_divide] = season_stats[cols_to_divide].div(season_stats['games'].values, axis=0)
season_stats = season_stats.rename(columns={'totalYards': 'avgYards'}) 
season_stats

In [347]:
# Export to csv and check it works
season_stats.to_csv('season_stats.csv', index=False)
test_season_stats = pd.read_csv('season_stats.csv')
test_season_stats

,team,season,conference,firstDowns,fourthDownConversions,fourthDowns,fumblesLost,fumblesRecovered,games,interceptionTDs,...,puntReturns,rushingAttempts,rushingTDs,rushingYards,sacks,tacklesForLoss,thirdDownConversions,thirdDowns,avgYards,turnovers
0,Air Force,2015,Mountain West,21.428571,1.000000,1.642857,0.785714,0.500000,14.0,1.0,...,1.071429,58.285714,3.214286,319.142857,NaN,NaN,6.000000,14.000000,451.071429,1.642857
1,Air Force,2016,Mountain West,23.230769,0.615385,1.153846,0.692308,0.615385,13.0,1.0,...,0.769231,63.615385,3.000000,317.384615,2.307692,5.461538,7.615385,15.076923,452.153846,1.307692
2,Air Force,2017,Mountain West,24.166667,1.333333,2.166667,1.166667,0.333333,12.0,1.0,...,1.000000,63.833333,3.166667,306.916667,0.583333,3.166667,6.750000,13.916667,417.000000,1.666667
3,Air Force,2018,Mountain West,22.000000,2.000000,3.250000,0.833333,0.583333,12.0,2.0,...,1.000000,59.416667,2.750000,283.666667,1.000000,4.250000,5.833333,14.083333,415.000000,1.250000
4,Air Force,2019,Mountain West,22.000000,1.230769,1.615385,0.923077,1.076923,13.0,3.0,...,0.384615,58.076923,3.153846,298.538462,2.461538,5.615385,6.461538,12.076923,421.769231,1.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,Wyoming,2016,Mountain West,22.285714,0.714286,1.285714,0.642857,0.785714,14.0,3.0,...,2.285714,44.857143,2.214286,204.642857,2.142857,5.642857,6.285714,14.428571,433.714286,1.714286
770,Wyoming,2017,Mountain West,15.230769,0.692308,1.076923,0.461538,1.307692,13.0,2.0,...,1.846154,34.307692,1.000000,108.769231,2.461538,6.692308,4.615385,14.076923,286.000000,1.076923
771,Wyoming,2018,Mountain West,17.000000,0.416667,1.083333,0.583333,0.416667,12.0,1.0,...,2.250000,41.250000,1.583333,199.583333,1.500000,5.500000,5.250000,14.500000,330.916667,0.916667
772,Wyoming,2019,Mountain West,17.692308,0.461538,0.923077,0.461538,0.923077,13.0,2.0,...,2.615385,44.230769,2.076923,214.769231,3.384615,8.000000,6.230769,14.461538,350.923077,1.153846


## Returning Team Production

In [251]:
prod_url = base_url + 'player/returning'

In [259]:
production_df = pd.DataFrame()
for year in years:
    response = my_session.get(prod_url, params={'year':year})
    production_df = production_df.append(response.json())
production_df.sort_values(['team', 'season'], inplace=True)

In [ ]:
production_df

Well, there's not really much to do here. The `totalPPA` column entails a team's [Total Predicted Points Added](https://collegefootballdata.com/Glossary#ExpectedPointsAdded), which is a measure of the team's returning player production

In [345]:
production_df[['season', 'team', 'conference', 'totalPPA']].to_csv('production.csv', index=False)

In [346]:
test_production = pd.read_csv('production.csv')
test_production

,season,team,conference,totalPPA
0,2015,Air Force,Mountain West,204.1
1,2016,Air Force,Mountain West,207.7
2,2017,Air Force,Mountain West,207.4
3,2018,Air Force,Mountain West,192.1
4,2019,Air Force,Mountain West,206.4
...,...,...,...,...
769,2016,Wyoming,Mountain West,187.3
770,2017,Wyoming,Mountain West,225.0
771,2018,Wyoming,Mountain West,110.7
772,2019,Wyoming,Mountain West,102.1


## Game Results

In [266]:
results_url = base_url + 'games'

'https://api.collegefootballdata.com/games'

In [267]:
results_df = pd.DataFrame()
for year in years:
    response = my_session.get(results_url, params={'year':year})
    results_df = results_df.append(response.json())

,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,...,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
0,400603840,2015,1,regular,2015-09-03T18:00:00.000Z,None,True,False,51664.0,3628,...,17.0,"[0, 10, 0, 7]",0.32245932505239777,153,North Carolina,ACC,13.0,"[7, 6, 0, 0]",0.6775406749476023,7.94978585869999
1,400763593,2015,1,regular,2015-09-03T18:00:00.000Z,None,False,False,39184.0,3652,...,14.0,"[7, 7, 0, 0]",0.10131731322441452,2229,Florida International,Conference USA,15.0,"[3, 0, 6, 6]",0.8986826867755855,5.3903379073
2,400787299,2015,1,regular,2015-09-03T19:00:00.000Z,None,False,False,10473.0,3919,...,48.0,"[3, 17, 14, 14]",0.5333798288849951,2678,VMI,None,36.0,"[0, 10, 7, 19]",0.46662017111500487,4.7380781049
3,400756896,2015,1,regular,2015-09-03T19:00:00.000Z,None,False,False,27126.0,3630,...,41.0,"[3, 21, 14, 3]",0.9999861779726812,2210,Elon,None,3.0,"[0, 0, 3, 0]",0.000013822027318810548,2.0872957646
4,400763399,2015,1,regular,2015-09-03T19:00:00.000Z,None,False,False,19717.0,3786,...,13.0,"[3, 3, 7, 0]",0.08629501978357867,197,Oklahoma State,Big 12,24.0,"[3, 7, 7, 7]",0.9137049802164213,3.1885406359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,401269381,2020,16,regular,2020-12-19T22:30:00.000Z,False,False,True,None,3632,...,56.0,"[21, 21, 7, 7]",0.9987953386468736,356,Illinois,Big Ten,21.0,"[21, 0, 0, 0]",0.0012046613531263572,2.3321812097
541,401269376,2020,16,regular,2020-12-20T00:00:00.000Z,False,False,True,None,1056,...,47.0,"[3, 0, 14, 17, 7, 6]",0.9336367489306686,24,Stanford,Pac-12,48.0,"[7, 13, 0, 14, 7, 7]",0.06636325106933139,8.4097108547
542,401237074,2020,16,regular,2020-12-20T01:00:00.000Z,False,True,True,None,5348,...,46.0,"[10, 7, 14, 15]",0.21734787364033173,333,Alabama,SEC,52.0,"[14, 21, 0, 17]",0.7826521263596683,3.2077818297
543,401264085,2020,16,regular,2020-12-20T01:00:00.000Z,False,False,True,None,3854,...,27.0,"[10, 7, 7, 3]",0.9051018477170518,202,Tulsa,American Athletic,24.0,"[0, 10, 7, 7]",0.09489815228294818,5.2550787629


Here we'll create a feature detailing if the home or away team won. We'll later turn this into a binary response for if the home team won, which will be our model's target variable.

In [269]:
def winner(df):
    if df['away_points'] > df['home_points']:
        return 'A'
    else:
        return 'H'
results_df['winner'] = results_df.apply(winner, axis=1)

In [276]:
results_df.columns

Index(['id', 'season', 'week', 'season_type', 'start_date', 'start_time_tbd',
       'neutral_site', 'conference_game', 'attendance', 'venue_id', 'venue',
       'home_id', 'home_team', 'home_conference', 'home_points',
       'home_line_scores', 'home_post_win_prob', 'away_id', 'away_team',
       'away_conference', 'away_points', 'away_line_scores',
       'away_post_win_prob', 'excitement_index', 'winner'],
      dtype='object')

In [350]:
columns_to_keep = ['id', 'season',
                   'home_team', 'home_conference',
                   'away_team', 'away_conference',
                   'winner']
results_df[columns_to_keep].to_csv('results.csv', index=False)

In [351]:
test_results = pd.read_csv('results.csv')
test_results

,id,season,home_team,home_conference,away_team,away_conference,winner
0,400603840,2015,South Carolina,SEC,North Carolina,ACC,H
1,400763593,2015,UCF,American Athletic,Florida International,Conference USA,A
2,400787299,2015,Ball State,Mid-American,VMI,NaN,H
3,400756896,2015,Wake Forest,ACC,Elon,NaN,H
4,400763399,2015,Central Michigan,Mid-American,Oklahoma State,Big 12,A
...,...,...,...,...,...,...,...
4728,401269381,2020,Penn State,Big Ten,Illinois,Big Ten,H
4729,401269376,2020,UCLA,Pac-12,Stanford,Pac-12,A
4730,401237074,2020,Florida,SEC,Alabama,SEC,A
4731,401264085,2020,Cincinnati,American Athletic,Tulsa,American Athletic,H


Now we can export these created files to a SQLite database for merging and further processing.